In [1]:
%matplotlib qt

In [2]:
#모델 yolo4
import tensorflow as tf
import os

def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    return model_path

model = tf.saved_model.load(download_model('metrabs_rn18_y4'))
    

In [3]:
#draw
import cv2
def loop_through_people(frame ,keypoints_with,boxes_with,check):

    for person, box in zip(keypoints_with,boxes_with):
        a,b,c,d,per = box
        
        
        if per > 0.50:    
            for xy in person:
                cv2.circle(frame,(int(xy[0]),int(xy[1])), 3, (0,255,0),-1)
                cv2.rectangle(frame,(int(a),int(b),int(c),int(d)),(255,0,0),2) 
                p = round(per,5)
                cv2.putText(frame,str(p),(int(a),int(b)),1,1,(0,255,0),2,cv2.LINE_AA)

In [4]:
# 디버깅
import cv2
import pandas as pd


cam1=cv2.VideoCapture("C:/Users/Leeyourack/jookyoung/video/1.test_video/time_test1.avi")
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('C:/Users/Leeyourack/jookyoung/video/2.detect_video/TEST.mp4', fourcc, 5.0, (2560, 720))
frame=0
df= []
detect=[]
while True:
    
    #영상 로드
    _,img1=cam1.read()
    _,img2=cam1.read()

    '''#영상 정보
    w = round(cam1.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cam1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cam1.get(cv2.CAP_PROP_FPS)'''
    

    #이미지 분석
    results1 = model.detect_poses(img1, skeleton='smpl_24') #예측1
    
    poses2d_1 = results1['poses2d'].numpy()
    poses3d_1 = results1['poses3d'].numpy()
    boxes_1 = results1['boxes'].numpy()

    
    #keypoints추출
    loop_through_people(img1, poses2d_1, boxes_1,frame)
    
    #데이터 프레임 만들기
    df.append([frame,'nan'])
    for box in boxes_1:
        a,b,c,d,per = box
        if per > 0.50:
            df[frame][1]=per
    

    


    #이미지 합치기
    cv2.resize(img1,(480,640))
    cv2.resize(img2,(480,640))

    two_img=cv2.hconcat([img1,img2])
    


    #디버깅을 위한 이미지 식별번호
    cv2.putText(two_img, f"{frame}" , (900,100), cv2.FONT_HERSHEY_SIMPLEX, 4, (0,255,0), 2)
    



    #이미지 보여주기
    cv2.imshow("TEST1",two_img)

    

    #이미지 저장하기
    cv2.imwrite('C:/Users/Leeyourack/jookyoung/image/ar1/''3d'+str(frame)+'.jpg',two_img)

    #영상 저장하기
    out.write(two_img)

    
    frame = frame + 1
    
    if cv2.waitKey(1) == ord('q'):
        break

    df_frame= pd.DataFrame(df, columns = ['frame','percent'])
    df_frame.to_csv('C:/Users/Leeyourack/jookyoung/data/debug.csv')

    
cam1.release() 
out.release()
cv2.destroyAllWindows()

In [5]:
#이미지 영상으로 만들기

import cv2
import numpy as np
import glob
 
img_array = []


 
for filename in [f"3d{1*(x+1)}.jpg" for x in range(1916)]:
    img = cv2.imread(f"C:/Users/Leeyourack/jookyoung/image/ar1/{filename}")
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('C:/Users/Leeyourack/jookyoung/video/2.detect_video/test11.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
